### Creating the main table for part 3 (from part 1 and 2)

In [1]:
#import the libraries BeautifulSoup and io and pandas
import bs4 as bs
from bs4 import BeautifulSoup
import io
import urllib.request
import pandas as pd

#importing more libraries
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Importing libraries and APIs to convert a neighbourhood and borough into latitude and longitude values and create dataframes
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library for JSON files
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # to get latitude and longitude values
import requests # it handles requests
from pandas.io.json import json_normalize # from JSON file to a pandas dataframe

#Creating the source from which to scrape the data from the table called "List of postal codes of Canada: M" from thr wikipedia page
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()

#Creating table object with BeautifulSoup
soup=BeautifulSoup(source, 'lxml')
table=soup.table

#Scraping the columns' names from "List of postal codes of Canada: M" from thr wikipedia
table_header=table.find_all('th')
for th in table_header:
    headers=[]
    headers.append(th.text)

# Creating a table with pandas that contains the data scraped from "List of postal codes of Canada: M" from thr wikipedia
table_rows=table.find_all('tr')
list_of_rows=[]
for tr in table_rows:
    td=tr.find_all('td')
    row = [item.text.strip('\n') for item in td]
    list_of_rows.append(row)
    
df=pd.DataFrame(list_of_rows, columns=["Postcode", "Borough", "Neighbourhood"]) 
df=df.iloc[1:]

#Keeping the strings for the column 'Borough' that are different from 'Not assigned'
df=df[df['Borough'] != 'Not assigned']
df=df.iloc[:].reset_index(drop=True)

#The 'Not assigned' values in the column 'Neighbourhood' are overwritten with the corresponding string in the column 'Borough'
for i, row in df.iterrows():
    if df.loc[i,'Neighbourhood'] == 'Not assigned':
        df.loc[i,'Neighbourhood'] = df.loc[i, 'Borough'] 
        
#Rows of the dataframe are grouped by 'Postcode' and 'Postcode' and
# the corresponding 'Neighbourhoods are combined into one row with the neighborhoods separated with a comma 
df_1 = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: x.str.cat(sep=', '))

#Table from part 1 that I am going to use in this notebook (part2)
df_final_part_1= pd.DataFrame(df_1).reset_index(drop=False)

#Getting the geospatial data (longitude, latitude and postcode) in a dataframe (table)
url="http://cocl.us/Geospatial_data"
source_toronto=requests.get(url).content
long_lat_postcode_df=pd.read_csv(io.StringIO(source_toronto.decode('utf-8')))

#Adding latitude and longitude to the dataframe obtained from part_1
Table_part_2 = pd.merge(df_final_part_1,
                        long_lat_postcode_df,
                        left_on='Postcode',
                        right_on='Postal Code',
                        how='left')

#Dropping the duplicate column 'postal code' (it's like Postcode)
#Final Table for Part 2 of the IBM project
Table_part_2.drop(['Postal Code'], axis=1)


Solving environment: ...working... done

# All requested packages already installed.



,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [2]:
#To have an idea of the number of neighbourhoods and boroughs in Toronto.
print('Toronto has {} boroughs and {} neighbourhoods.'.format(
        len(Table_part_2['Borough'].unique()),
        len(Table_part_2['Neighbourhood'].unique())
         )
     )

Toronto has 11 boroughs and 103 neighbourhoods.


In [3]:
#Toronto latitude and longitude using the geopy library
address = 'Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Visualizing neighborhoods on Toronto's map.

In [4]:
# For Toronto I am using the same method used for New York and I also import Folium for map visualization of the 
!pip install folium
import folium 

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Table_part_2['Latitude'], Table_part_2['Longitude'], Table_part_2['Borough'], Table_part_2['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

###  Exploring Scarborough borough in Toronto with Foursquare API and segment them.

In [5]:
CLIENT_ID = 'CZYEM15LS4UA5JZQQST5BNODI5RHDKMCNZAKQVDDIYOWJ1KB' # my Foursquare ID
CLIENT_SECRET = 'KBY2IB4DOTCOS2BJNT32P1WX2OXKNAO2WO3VVHUWHRDUIUMX' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CZYEM15LS4UA5JZQQST5BNODI5RHDKMCNZAKQVDDIYOWJ1KB
CLIENT_SECRET:KBY2IB4DOTCOS2BJNT32P1WX2OXKNAO2WO3VVHUWHRDUIUMX


In [6]:
Table_part_2.loc[0, 'Borough'] #exploring the 1st borough of the dataframe Table_part_2

'Scarborough'

In [7]:
scarborough_df = Table_part_2[Table_part_2['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_df.head(2)

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497


In [8]:
borough_latitude = scarborough_df.loc[0, 'Latitude'] # neighborhood latitude value
borough_longitude = scarborough_df.loc[0, 'Longitude'] # neighborhood longitude value

borough_name = scarborough_df.loc[0, 'Borough'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(borough_name, 
                                                               borough_latitude, 
                                                            borough_longitude))

Latitude and longitude values of Scarborough are 43.806686299999996, -79.19435340000001.


### Creating a map of Scarborough

In [9]:
#Scarborough latitude and longitude using the geopy library
address = 'Scarborough, Toronto'
geolocator = Nominatim(user_agent="scarborough_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [10]:
address_scarbour = 'Scarborough'
latitude_scarbour = latitude
longitude_scarbour = longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scarbour, longitude_scarbour))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [11]:
# create map of Toronto using latitude and longitude values
map_scarbour = folium.Map(location=[latitude_scarbour, longitude_scarbour], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(scarborough_df['Latitude'], scarborough_df['Longitude'], scarborough_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarbour)  
    
map_scarbour

#### Top 100 venues that are in Scarborough within a radius of 1000 meters

In [12]:
#setting 100 venues and 500 meters radius
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    'CZYEM15LS4UA5JZQQST5BNODI5RHDKMCNZAKQVDDIYOWJ1KB', 
    'KBY2IB4DOTCOS2BJNT32P1WX2OXKNAO2WO3VVHUWHRDUIUMX', 
    '20180605' , 
    latitude_scarbour, 
    longitude_scarbour, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=CZYEM15LS4UA5JZQQST5BNODI5RHDKMCNZAKQVDDIYOWJ1KB&client_secret=KBY2IB4DOTCOS2BJNT32P1WX2OXKNAO2WO3VVHUWHRDUIUMX&v=20180605&ll=43.773077,-79.257774&radius=500&limit=100'

In [13]:
print(" Scarborough_latitude is: {},  Scarborough_longitude is: {}".format(latitude_scarbour, longitude_scarbour))

 Scarborough_latitude is: 43.773077,  Scarborough_longitude is: -79.257774


In [14]:
#Examining the results of the GET request 
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d90e32a48b1e1002bd0afa0'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Scarborough City Centre',
  'headerFullLocation': 'Scarborough City Centre, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 44,
  'suggestedBounds': {'ne': {'lat': 43.7775770045, 'lng': -79.25155367954714},
   'sw': {'lat': 43.7685769955, 'lng': -79.26399432045285}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5085ec39e4b0b1ead2eb0818',
       'name': 'Disney Store',
       'location': {'address': '300 Borough Drive',
        'crossStreet': 'in Scarborough Town Centre',
        'lat': 43.775537,
        'lng': -79.256833,
        'labeledLa

In [15]:
# From Foursquare API function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Top 100 venues in Scarborough

In [16]:

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,American Eagle Outfitters,Clothing Store,43.775908,-79.258352
2,SEPHORA,Cosmetics Shop,43.775017,-79.258109
3,DAVIDsTEA,Tea Room,43.776613,-79.258516
4,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
5,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
6,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
7,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
8,Shoppers Drug Mart,Pharmacy,43.773305,-79.251662
9,Hot Topic,Clothing Store,43.775450,-79.257929


In [17]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

44 venues were returned by Foursquare.


In [18]:
#Function to get the nearby venues (radius=500 meters, names:neighborhood) 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Venues are in the Scarborough's neighborhoods

In [19]:
#Which venues are in the Scarborough's neighborhoods (radius = 500 m)? The list of neighborhood is given by the getNearbyVenues function
#applied to the scarborough_df dataframe
scarborough_nearby_venues = getNearbyVenues(names=scarborough_df['Neighbourhood'],
                                   latitudes=scarborough_df['Latitude'],
                                   longitudes=scarborough_df['Longitude']
                                           )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge


In [20]:
#Getting the evnues in Scarborough's nearby neighborhoods (radius 500 meters)
scarborough_nearby_venue_df=pd.DataFrame(scarborough_nearby_venues)
scarborough_nearby_venue_df

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
5,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
6,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
7,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection
8,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot
9,Woburn,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop


In [21]:
#Checking the size of the resulting dataframe
print(scarborough_nearby_venue_df.shape)

(91, 7)


### Exploring the venues' dataset for neighborhoods nearby Scarborough

In [22]:
#Number of Venues for neighborhood
scarborough_nearby_venue_df.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,7,7,7,7,7,7
"Clairlea, Golden Mile, Oakridge",9,9,9,9,9,9
"Clarks Corners, Sullivan, Tam O'Shanter",13,13,13,13,13,13
"Cliffcrest, Cliffside, Scarborough Village West",3,3,3,3,3,3
"Dorset Park, Scarborough Town Centre, Wexford Heights",7,7,7,7,7,7
"East Birchmount Park, Ionview, Kennedy Park",6,6,6,6,6,6


In [23]:
#Unique neighbourhoods in Scarborough
scarborough_nearby_venue_df['Neighborhood'].unique().tolist()


['Rouge, Malvern',
 'Highland Creek, Rouge Hill, Port Union',
 'Guildwood, Morningside, West Hill',
 'Woburn',
 'Cedarbrae',
 'Scarborough Village',
 'East Birchmount Park, Ionview, Kennedy Park',
 'Clairlea, Golden Mile, Oakridge',
 'Cliffcrest, Cliffside, Scarborough Village West',
 'Birch Cliff, Cliffside West',
 'Dorset Park, Scarborough Town Centre, Wexford Heights',
 'Maryvale, Wexford',
 'Agincourt',
 "Clarks Corners, Sullivan, Tam O'Shanter",
 "Agincourt North, L'Amoreaux East, Milliken, Steeles East",
 "L'Amoreaux West"]

In [24]:
#Number of unique neighbourhoods in Scarborough
print("The number of neighborhood in Scarborough, borough in Toronto is: {}.".format(len(scarborough_nearby_venue_df['Neighborhood'].unique().tolist())))

The number of neighborhood in Scarborough, borough in Toronto is: 16.


In [25]:
#Unique venue category in Scarborough
scarborough_nearby_venue_df['Venue Category'].unique().tolist()

['Fast Food Restaurant',
 'Bar',
 'Pizza Place',
 'Electronics Store',
 'Mexican Restaurant',
 'Rental Car Location',
 'Medical Center',
 'Intersection',
 'Breakfast Spot',
 'Coffee Shop',
 'Korean Restaurant',
 'Hakka Restaurant',
 'Caribbean Restaurant',
 'Thai Restaurant',
 'Athletics & Sports',
 'Bank',
 'Bakery',
 'Fried Chicken Joint',
 'Playground',
 'Grocery Store',
 'Department Store',
 'Convenience Store',
 'Discount Store',
 'Hobby Shop',
 'Bus Line',
 'Metro Station',
 'Bus Station',
 'Park',
 'Soccer Field',
 'Motel',
 'Movie Theater',
 'American Restaurant',
 'Café',
 'General Entertainment',
 'Skating Rink',
 'College Stadium',
 'Indian Restaurant',
 'Chinese Restaurant',
 'Latin American Restaurant',
 'Pet Store',
 'Vietnamese Restaurant',
 'Light Rail Station',
 'Sandwich Place',
 'Middle Eastern Restaurant',
 'Shopping Mall',
 'Auto Garage',
 'Lounge',
 'Clothing Store',
 'Italian Restaurant',
 'Noodle House',
 'Pharmacy',
 'Sculpture Garden',
 'Bubble Tea Shop']

In [26]:
#Number of unique neighbourhoods in Scarborough
print("The number of unique venues in Scarborough's nearby neighborhoods (Toronto) is: {}.".format(len(scarborough_nearby_venue_df['Venue Category'].unique().tolist())))

The number of unique venues in Scarborough's nearby neighborhoods (Toronto) is: 53.


In [27]:
#Number of venues' categories in Scarborough's neighborhoods
most_frequent_venue_cat_scarborough=scarborough_nearby_venue_df.groupby(['Venue Category']).count()
venue_count=scarborough_nearby_venue_df.groupby(['Venue Category']).count()
venue_count=venue_count['Neighborhood'].tolist()
most_frequent_venue_cat_scarborough['Venue_count']=venue_count
venues_scar_count=most_frequent_venue_cat_scarborough.drop(['Neighborhood','Neighborhood Latitude','Neighborhood Longitude','Venue', 'Venue Latitude', 'Venue Longitude'], axis=1)
venues_scar_count=venues_scar_count.reset_index(drop=False)
venues_scar_count.rename(columns={'Venue Category':'Category'}, inplace=True)
venues_scar_count

,Category,Venue_count
0,American Restaurant,1
1,Athletics & Sports,1
2,Auto Garage,1
3,Bakery,4
4,Bank,2
5,Bar,1
6,Breakfast Spot,5
7,Bubble Tea Shop,1
8,Bus Line,1
9,Bus Station,1


In [28]:
# Visualising Venues category count in Scorborough in Toronto with a basic barplot. There are lots of food stores and bars!!
import matplotlib.pyplot as plt
import numpy as np

bars = pd.Series(venues_scar_count.Category)
height = pd.Series(venues_scar_count.Venue_count)
y_pos = np.arange(len(bars))
plt.bar(y_pos, height)
 
# If we have long labels, we cannot see it properly
names=venues_scar_count.Category.tolist()
plt.xticks(y_pos, names, rotation=90)
 
# Thus we have to give more margin:
plt.subplots_adjust(bottom=0.4)
 
# It's the same concept if you need more space for your titles
plt.title("Venues in Scorborough's neighborhoods, Toronto")
plt.subplots_adjust(top=0.7)



### Analyzing Each Neighborhood in Scarborough

In [29]:
#Getting back to the scarborough's nearby neighborhoods venues dataset
scarborough_nearby_venue_df=pd.DataFrame(scarborough_nearby_venues)
scarborough_nearby_venue_df

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
5,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
6,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
7,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection
8,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot
9,Woburn,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop


In [30]:
# one hot encoding
scarb_onehot = pd.get_dummies(scarborough_nearby_venue_df[['Venue']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['Neighborhood'] = scarborough_nearby_venue_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighborhood,A&W Canada,B&A Bakery,Bakery On The Go,Big Al's Pet Supercentre,Big Bite Burrito,Birchmount Community Centre,Birchmount Stadium,Black Pearl Remnant,Bros. CONVENIENCE,Cafe on the go,Centennial Recreation Centre,Clairlea Futbol Centre,Coffee Time,Commander Arena,Crown Pastries,Dairy Queen,Dollarama,Drupati's Roti & Doubles,Eaton Centre USA,Eggsmart,Eight Noodles,El Pulgarcito,Enterprise Rent-A-Car,Federick Restaurant,G & G Electronics,Giant Tiger,Gusto Pizza,Have A Nap Motel,Jenns Den,KFC,Kairali,Karaikudi Chettinad South Indian Restaurant,Kim Kim restaurant,Korean Grill House,Kub Khao,Lawrence Ave E & Kingston Rd,Lebanese bakery,Little Caesars,Mark's,McCowan Park,McDonald's,Milliken Public School Playground,Mr Congee Chinese Cuisine 龍粥記,Panagio's Breakfast & Lunch,Pho Vietnam,Phoenix Restaurant 金鳳餐廳,Pizza Pizza,Popeyes Louisiana Kitchen,Port Royal Park,Presotea,Price Chopper,Remezzo Italian Bistro,Rexall,Rogers Plus,Royal Canadian Legion,Scarborough Garage Door Repair,Scarborough Gardens,Scarborough LRT,Sequoia Lounge,Shoppers Drug Mart,Starbucks,Subway,Super Taste Noodle House,Swiss Chalet Rotisserie & Grill,TD Canada Trust,TTC Bus #68 Warden,Tandy Leather,Thai One On,The Birchcliff,The Royal Chinese Restaurant 避風塘小炒,Tim Hortons,Tumbe Cafe,Twilight,Vincent's Spot,Warden Ave & St. Clair Ave E,Warden Station Bus Loop,Warden Subway Station,Warden Woods Park,Wendy's,Wexford Heights Plaza,Wexford Restaurant,Woburn Medical Centre
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
# examining the new dataframe scarb_onehot size
scarb_onehot.size

7553

### Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [32]:
scarborough_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,A&W Canada,B&A Bakery,Bakery On The Go,Big Al's Pet Supercentre,Big Bite Burrito,Birchmount Community Centre,Birchmount Stadium,Black Pearl Remnant,Bros. CONVENIENCE,Cafe on the go,Centennial Recreation Centre,Clairlea Futbol Centre,Coffee Time,Commander Arena,Crown Pastries,Dairy Queen,Dollarama,Drupati's Roti & Doubles,Eaton Centre USA,Eggsmart,Eight Noodles,El Pulgarcito,Enterprise Rent-A-Car,Federick Restaurant,G & G Electronics,Giant Tiger,Gusto Pizza,Have A Nap Motel,Jenns Den,KFC,Kairali,Karaikudi Chettinad South Indian Restaurant,Kim Kim restaurant,Korean Grill House,Kub Khao,Lawrence Ave E & Kingston Rd,Lebanese bakery,Little Caesars,Mark's,McCowan Park,McDonald's,Milliken Public School Playground,Mr Congee Chinese Cuisine 龍粥記,Panagio's Breakfast & Lunch,Pho Vietnam,Phoenix Restaurant 金鳳餐廳,Pizza Pizza,Popeyes Louisiana Kitchen,Port Royal Park,Presotea,Price Chopper,Remezzo Italian Bistro,Rexall,Rogers Plus,Royal Canadian Legion,Scarborough Garage Door Repair,Scarborough Gardens,Scarborough LRT,Sequoia Lounge,Shoppers Drug Mart,Starbucks,Subway,Super Taste Noodle House,Swiss Chalet Rotisserie & Grill,TD Canada Trust,TTC Bus #68 Warden,Tandy Leather,Thai One On,The Birchcliff,The Royal Chinese Restaurant 避風塘小炒,Tim Hortons,Tumbe Cafe,Twilight,Vincent's Spot,Warden Ave & St. Clair Ave E,Warden Station Bus Loop,Warden Subway Station,Warden Woods Park,Wendy's,Wexford Heights Plaza,Wexford Restaurant,Woburn Medical Centre
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.0,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.333333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,Cedarbrae,0.000000,0.142857,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.00

In [33]:
#checking the size
scarborough_grouped.size

1328

### Ten top venues per neighborhood in Scarborough

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Mark's,Panagio's Breakfast & Lunch,Commander Arena,Twilight,KFC,Enterprise Rent-A-Car,Federick Restaurant,G & G Electronics,Giant Tiger,Gusto Pizza
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Port Royal Park,Milliken Public School Playground,Black Pearl Remnant,Kairali,Federick Restaurant,G & G Electronics,Giant Tiger,Gusto Pizza,Have A Nap Motel,Jenns Den
2,"Birch Cliff, Cliffside West",Scarborough Gardens,Birchmount Community Centre,Birchmount Stadium,The Birchcliff,Woburn Medical Centre,KFC,Federick Restaurant,G & G Electronics,Giant Tiger,Gusto Pizza
3,Cedarbrae,Centennial Recreation Centre,B&A Bakery,Federick Restaurant,Drupati's Roti & Doubles,TD Canada Trust,Popeyes Louisiana Kitchen,Thai One On,Jenns Den,G & G Electronics,Giant Tiger
4,"Clairlea, Golden Mile, Oakridge",Dairy Queen,TTC Bus #68 Warden,Bakery On The Go,Warden Woods Park,Warden Subway Station,Warden Station Bus Loop,Warden Ave & St. Clair Ave E,Clairlea Futbol Centre,Cafe on the go,Enterprise Rent-A-Car
5,"Clarks Corners, Sullivan, Tam O'Shanter",Rexall,A&W Canada,The Royal Chinese Restaurant 避風塘小炒,Kub Khao,KFC,Popeyes Louisiana Kitchen,Gusto Pizza,Remezzo Italian Bistro,Eight Noodles,Little Caesars
6,"Cliffcrest, Cliffside, Scarborough Village West",Vincent's Spot,Have A Nap Motel,Jenns Den,Woburn Medical Centre,KFC,El Pulgarcito,Enterprise Rent-A-Car,Federick Restaurant,G & G Electronics,Giant Tiger
7,"Dorset Park, Scarborough Town Centre, Wexford ...",Karaikudi Chettinad South Indian Restaurant,Scarborough LRT,Big Al's Pet Supercentre,Pho Vietnam,Kim Kim restaurant,El Pulgarcito,Kairali,Jenns Den,Federick Restaurant,G & G Electronics
8,"East Birchmount Park, Ionview, Kennedy Park",Dollarama,Tim Hortons,Giant Tiger,Tandy Leather,Bros. CONVENIENCE,KFC,Enterprise Rent-A-Car,Federick Restaurant,G & G Electronics,Gusto Pizza
9,"Guildwood, Morningside, West Hill",Woburn Medical Centre,Big Bite Burrito,Enterprise Rent-A-Car,Swiss Chalet Rotisserie & Grill,G & G Electronics,Eggsmart,Lawrence Ave E & Kingston Rd,Kub Khao,Korean Grill House,Kim Kim restaurant


### Cluster and label neighborhoods.
####  Here I set number of clusters equal to five and so the neighborhood will be labelled with an integers value (labels) ranging from 0 to 4 (included)

In [36]:
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)   #dropping the column 'Neighbourhood'

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_
print('kmeans_labels are the following array: {}, the number of labels are: {}'.format(kmeans.labels_, len(kmeans.labels_)))

kmeans_labels are the following array: [3 0 0 0 0 0 0 0 0 0 1 0 0 2 4 0], the number of labels are: 16


In [37]:
 scarborough_df  #This dataframe has 17 rows while the scarborough_grouped_clustering has only 16

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",M1K,43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",M1L,43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",M1M,43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",M1N,43.692657,-79.264848


In [38]:
neighborhoods_venues_sorted #This dataframe will ne joined to the scarborough_df but the number of rows is different. That's a problem.

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Mark's,Panagio's Breakfast & Lunch,Commander Arena,Twilight,KFC,Enterprise Rent-A-Car,Federick Restaurant,G & G Electronics,Giant Tiger,Gusto Pizza
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Port Royal Park,Milliken Public School Playground,Black Pearl Remnant,Kairali,Federick Restaurant,G & G Electronics,Giant Tiger,Gusto Pizza,Have A Nap Motel,Jenns Den
2,"Birch Cliff, Cliffside West",Scarborough Gardens,Birchmount Community Centre,Birchmount Stadium,The Birchcliff,Woburn Medical Centre,KFC,Federick Restaurant,G & G Electronics,Giant Tiger,Gusto Pizza
3,Cedarbrae,Centennial Recreation Centre,B&A Bakery,Federick Restaurant,Drupati's Roti & Doubles,TD Canada Trust,Popeyes Louisiana Kitchen,Thai One On,Jenns Den,G & G Electronics,Giant Tiger
4,"Clairlea, Golden Mile, Oakridge",Dairy Queen,TTC Bus #68 Warden,Bakery On The Go,Warden Woods Park,Warden Subway Station,Warden Station Bus Loop,Warden Ave & St. Clair Ave E,Clairlea Futbol Centre,Cafe on the go,Enterprise Rent-A-Car
5,"Clarks Corners, Sullivan, Tam O'Shanter",Rexall,A&W Canada,The Royal Chinese Restaurant 避風塘小炒,Kub Khao,KFC,Popeyes Louisiana Kitchen,Gusto Pizza,Remezzo Italian Bistro,Eight Noodles,Little Caesars
6,"Cliffcrest, Cliffside, Scarborough Village West",Vincent's Spot,Have A Nap Motel,Jenns Den,Woburn Medical Centre,KFC,El Pulgarcito,Enterprise Rent-A-Car,Federick Restaurant,G & G Electronics,Giant Tiger
7,"Dorset Park, Scarborough Town Centre, Wexford ...",Karaikudi Chettinad South Indian Restaurant,Scarborough LRT,Big Al's Pet Supercentre,Pho Vietnam,Kim Kim restaurant,El Pulgarcito,Kairali,Jenns Den,Federick Restaurant,G & G Electronics
8,"East Birchmount Park, Ionview, Kennedy Park",Dollarama,Tim Hortons,Giant Tiger,Tandy Leather,Bros. CONVENIENCE,KFC,Enterprise Rent-A-Car,Federick Restaurant,G & G Electronics,Gusto Pizza
9,"Guildwood, Morningside, West Hill",Woburn Medical Centre,Big Bite Burrito,Enterprise Rent-A-Car,Swiss Chalet Rotisserie & Grill,G & G Electronics,Eggsmart,Lawrence Ave E & Kingston Rd,Kub Khao,Korean Grill House,Kim Kim restaurant


In [39]:
#Checking whether 'Upper Rouge is present in 'Neighbourhood in the neighborhoods_venues_sorted dataframe. It's NOT.
(neighborhoods_venues_sorted['Neighborhood']=='Upper Rouge').any()  

False

In [40]:
# 'Upper Rouge is NOT present in 'Neighbourhood' column in the neighborhoods_venues_sorted dataframe so I can remove it from
#scarborough_df and join the two datasets that have now the same numbero of rows.
scarborough_merged = scarborough_df.drop(16)

# add clustering labels to the scarborough_nearby_venue_df
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

scarborough_merged

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353,3,Wendy's,Woburn Medical Centre,KFC,El Pulgarcito,Enterprise Rent-A-Car,Federick Restaurant,G & G Electronics,Giant Tiger,Gusto Pizza,Have A Nap Motel
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497,0,Royal Canadian Legion,Woburn Medical Centre,Eight Noodles,Enterprise Rent-A-Car,Federick Restaurant,G & G Electronics,Giant Tiger,Gusto Pizza,Have A Nap Motel,Jenns Den
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711,0,Woburn Medical Centre,Big Bite Burrito,Enterprise Rent-A-Car,Swiss Chalet Rotisserie & Grill,G & G Electronics,Eggsmart,Lawrence Ave E & Kingston Rd,Kub Khao,Korean Grill House,Kim Kim restaurant
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917,0,Starbucks,Tim Hortons,Korean Grill House,El Pulgarcito,Enterprise Rent-A-Car,Federick Restaurant,G & G Electronics,Giant Tiger,Gusto Pizza,Have A Nap Motel
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476,0,Centennial Recreation Centre,B&A Bakery,Federick Restaurant,Drupati's Roti & Doubles,TD Canada Trust,Popeyes Louisiana Kitchen,Thai One On,Jenns Den,G & G Electronics,Giant Tiger
5,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476,0,Tumbe Cafe,McCowan Park,Little Caesars,Jenns Den,El Pulgarcito,Enterprise Rent-A-Car,Federick Restaurant,G & G Electronics,Giant Tiger,Gusto Pizza
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",M1K,43.727929,-79.262029,0,Dollarama,Tim Hortons,Giant Tiger,Tandy Leather,Bros. CONVENIENCE,KFC,Enterprise Rent-A-Car,Federick Restaurant,G & G Electronics,Gusto Pizza
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",M1L,43.711112,-79.284577,0,Dairy Queen,TTC Bus #68 Warden,Bakery On The Go,Warden Woods Park,Warden Subway Station,Warden Station Bus Loop,Warden Ave & St. Clair Ave E,Clairlea Futbol Centre,Cafe on the go,Enterprise Rent-A-Car
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",M1M,43.716316,-79.239476,0,Vincent's Spot,Have A Nap Motel,Jenns Den,Woburn Medical Centre,KFC,El Pulgarcito,Enterprise Rent-A-Car,Federick Restaurant,G & G Electronics,Giant Tiger
9,M1N,Scarborough,"Birch Cliff, Cliffside West",M1N,43.692657,-79.264848,0,Scarborough Gardens,Birchmount Community Centre,Birchmount Stadium,The Birchcliff,Woburn Medical Centre,KFC,Federick Restaurant,G & G Electronics,Giant Tiger,Gusto Pizza


In [41]:
#Getting the columns of the merged dataframe
scarborough_merged.columns

Index(['Postcode', 'Borough', 'Neighbourhood', 'Postal Code', 'Latitude',
       'Longitude', 'Cluster Labels', '1st Most Common Venue',
       '2nd Most Common Venue', '3rd Most Common Venue',
       '4th Most Common Venue', '5th Most Common Venue',
       '6th Most Common Venue', '7th Most Common Venue',
       '8th Most Common Venue', '9th Most Common Venue',
       '10th Most Common Venue'],
      dtype='object')

### Scarborough neighborhoods' five clusters visualised on Toronto's map 

In [42]:
# create map
map_clusters = folium.Map(location = [latitude_scarbour, longitude_scarbour], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighbourhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Checking 1st cluster labelled 0. 

In [43]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,-79.160497,0,Royal Canadian Legion,Woburn Medical Centre,Eight Noodles,Enterprise Rent-A-Car,Federick Restaurant,G & G Electronics,Giant Tiger,Gusto Pizza,Have A Nap Motel,Jenns Den
2,Scarborough,-79.188711,0,Woburn Medical Centre,Big Bite Burrito,Enterprise Rent-A-Car,Swiss Chalet Rotisserie & Grill,G & G Electronics,Eggsmart,Lawrence Ave E & Kingston Rd,Kub Khao,Korean Grill House,Kim Kim restaurant
3,Scarborough,-79.216917,0,Starbucks,Tim Hortons,Korean Grill House,El Pulgarcito,Enterprise Rent-A-Car,Federick Restaurant,G & G Electronics,Giant Tiger,Gusto Pizza,Have A Nap Motel
4,Scarborough,-79.239476,0,Centennial Recreation Centre,B&A Bakery,Federick Restaurant,Drupati's Roti & Doubles,TD Canada Trust,Popeyes Louisiana Kitchen,Thai One On,Jenns Den,G & G Electronics,Giant Tiger
5,Scarborough,-79.239476,0,Tumbe Cafe,McCowan Park,Little Caesars,Jenns Den,El Pulgarcito,Enterprise Rent-A-Car,Federick Restaurant,G & G Electronics,Giant Tiger,Gusto Pizza
6,Scarborough,-79.262029,0,Dollarama,Tim Hortons,Giant Tiger,Tandy Leather,Bros. CONVENIENCE,KFC,Enterprise Rent-A-Car,Federick Restaurant,G & G Electronics,Gusto Pizza
7,Scarborough,-79.284577,0,Dairy Queen,TTC Bus #68 Warden,Bakery On The Go,Warden Woods Park,Warden Subway Station,Warden Station Bus Loop,Warden Ave & St. Clair Ave E,Clairlea Futbol Centre,Cafe on the go,Enterprise Rent-A-Car
8,Scarborough,-79.239476,0,Vincent's Spot,Have A Nap Motel,Jenns Den,Woburn Medical Centre,KFC,El Pulgarcito,Enterprise Rent-A-Car,Federick Restaurant,G & G Electronics,Giant Tiger
9,Scarborough,-79.264848,0,Scarborough Gardens,Birchmount Community Centre,Birchmount Stadium,The Birchcliff,Woburn Medical Centre,KFC,Federick Restaurant,G & G Electronics,Giant Tiger,Gusto Pizza
11,Scarborough,-79.295849,0,Lebanese bakery,Wexford Heights Plaza,Wexford Restaurant,Subway,Scarborough Garage Door Repair,Crown Pastries,Sequoia Lounge,Gusto Pizza,El Pulgarcito,Enterprise Rent-A-Car


## Checking 2nd cluster labelled 1

In [44]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,-79.273304,1,Karaikudi Chettinad South Indian Restaurant,Scarborough LRT,Big Al's Pet Supercentre,Pho Vietnam,Kim Kim restaurant,El Pulgarcito,Kairali,Jenns Den,Federick Restaurant,G & G Electronics


## Checking 3rd cluster labelled 2

In [45]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Scarborough,-79.304302,2,Rexall,A&W Canada,The Royal Chinese Restaurant 避風塘小炒,Kub Khao,KFC,Popeyes Louisiana Kitchen,Gusto Pizza,Remezzo Italian Bistro,Eight Noodles,Little Caesars


## Checking 4th cluster labelled 3

In [46]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,-79.194353,3,Wendy's,Woburn Medical Centre,KFC,El Pulgarcito,Enterprise Rent-A-Car,Federick Restaurant,G & G Electronics,Giant Tiger,Gusto Pizza,Have A Nap Motel


## Checking 5th cluster labelled 4

In [47]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,-79.284577,4,Port Royal Park,Milliken Public School Playground,Black Pearl Remnant,Kairali,Federick Restaurant,G & G Electronics,Giant Tiger,Gusto Pizza,Have A Nap Motel,Jenns Den


**Reference** 'Segmenting and Clustering Neighborhoods in New York City' by IBM